<a href="https://colab.research.google.com/github/elenagonzalezrebollo/AprendizajeAutomatico/blob/main/Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


https://www.kaggle.com/datasets/igmoncan/hoteles-en-valencia?resource=download&select=valenciahoteles.csv

In [11]:
data = pd.read_csv('valenciahoteles.csv', encoding='latin-1')
data

,Nombre,Distrito,Valoración,Número de comentarios,Precio,Fecha,Dirección,CP,lat,long
0,HOSTAL ABASTOS,"Extramurs, Valencia",7.9,767,38.0,2022-07-31,"CALLE DE ALBERIQUE, 19, Valencia",46008,39.466484,-0.388092
1,BIZZBEACH,"Poblados marítimos, Valencia",8.0,206,37.0,2022-07-31,"CALLE DE IBIZA, 17, Valencia",46024,39.460211,-0.339943
2,HOSTAL ON DORMIRE VALENCIA,"Quatre Carreres, Valencia",7.4,1458,27.5,2022-07-31,"CALLE DE RUBEN VELA, 23, Valencia",46006,39.455528,-0.374437
3,ZALAMERA BNB,"Extramurs, Valencia",8.8,1009,79.0,2022-07-31,"CALLE DE PELAYO, 44, LA ROQUETA, Valencia",46007,39.465586,-0.379132
4,SWEET HOTEL RENASA,"El Pla del Real, Valencia",8.1,3441,84.0,2022-07-31,"AVENIDA DE CATALUÑA, 5, Valencia",46010,39.480567,-0.352960
...,...,...,...,...,...,...,...,...,...,...
3954,SH SUITE PALACE,"Ciutat Vella, Valencia",8.5,261,105.5,2022-11-13,"PLAÇA DE MARGARITA VALLDAURA, 2, Valencia",46002,39.473334,-0.374854
3955,LOTELITO,"Ciutat Vella, Valencia",8.8,1299,228.0,2022-11-13,"CALLE DE VIRUÉS, 6, Valencia",46002,39.470981,-0.374681
3956,ORIGINAL DOMINO HOUSE,"Camins al Grau, Valencia",8.7,725,102.0,2022-11-13,"CALLE DE LAS ISLAS CANARIAS, 51, GRAO, Valencia",46023,39.461333,-0.338056
3957,REINA 107 BEACH BOUTIQUE,"Poblados marítimos, Valencia",8.7,393,349.0,2022-11-13,"CARRER DE LA REINA, 107, Valencia",46011,39.468452,-0.328803
